In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import PIL
from tqdm import tqdm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#train_dir = sorted(glob("../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/*.dcm"))
#test_dir = sorted(glob("../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/*.dcm"))
#print("train files: ", len(train_dir))
#print("test files: ", len(test_dir))

In [ ]:
train_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv')
sub_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv')

train_df['filename'] = train_df['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".dcm")
train_df['type'] = train_df['ID'].apply(lambda st: st.split('_')[2])

In [ ]:
np.random.seed(1749)
sample_files = np.random.choice(os.listdir('/kaggle/input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'), 100)

sample_df = train_df[train_df.filename.isin(sample_files)]

In [ ]:
pivot_df = sample_df[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()

In [ ]:

def save_as_dicom(filenames, load_dir):    
    save_dir = '/kaggle/tmp/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for filename in tqdm(filenames):
        path = load_dir + filename
        new_path = save_dir + filename
        
        dcm = pydicom.dcmread(path)
        img = dcm.pixel_array
        pydicom.filewriter.write_file(new_path, img, write_like_original=True)

In [ ]:
save_as_dicom(filenames=sample_files, load_dir='/kaggle/input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/')

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(validation_split=0.15)
dataset = datagen.flow_from_dataframe(
        pivot_df, 
        directory='/kaggle/tmp/',
        x_col='filename', 
        y_col=['any', 'epidural', 'intraparenchymal', 
               'intraventricular', 'subarachnoid', 'subdural'],
        class_mode='multi_output',
        target_size=(512, 512),
        batch_size=100
    )

In [ ]:
dataset_size = 100

# create model
model = Sequential()
model.add(Dense(32, activation='tanh', input_shape=(512, 512)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics= 
['accuracy'])
model.fit(dataset, pivot_df, epochs=5, shuffle=True, batch_size=20)